In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import copy 
import math 
from utils import get_smi_list, replace_atom, get_dic, encode_smi, pad_smi, clones, parallel_f, pad, normalize, get_atom_pos, MyDataset, subsequent_mask
from model import Encoder, Decoder, device
from tqdm.auto import tqdm


In [2]:
smi_list = get_smi_list('data/ADAGRASIB_SMILES.txt')

coor_list = parallel_f(get_atom_pos, smi_list)
longest_coor = len(max(coor_list, key = len))
coor_list = [pad(normalize(c), longest_coor) for c in coor_list]

smi_list = [replace_atom(smi) for smi in smi_list]
smi_dic = get_dic(smi_list)
smint_list = [encode_smi(smi, smi_dic) for smi in smi_list]
longest_smint = len(max(smint_list, key = len))
smint_list = [pad_smi(smint, longest_smint, smi_dic) for smint in smint_list]

[13:43:23] UFFTYPER: Unrecognized atom type: Ba (0)


In [3]:
BATCH_SIZE = 1
dataset = MyDataset(smint_list, coor_list)
train_set, val_set, test_set = random_split(dataset, [0.9, 0.05, 0.05])

train_loader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_set, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_set, batch_size = BATCH_SIZE, shuffle = True)

In [6]:
DIM_MODEL = 256 
NUM_HEAD = 4
NUM_LAYER = 2
DROPOUT = 0.5

encoder = Encoder(DIM_MODEL, NUM_HEAD, NUM_LAYER, DROPOUT, len(smi_dic)).to(device)
decoder = Decoder(DIM_MODEL, NUM_HEAD, NUM_LAYER, DROPOUT, longest_coor).to(device)

loss_fn = nn.L1Loss() 
e_optim = torch.optim.Adam(encoder.parameters(), lr = 0.001)
d_optim = torch.optim.Adam(decoder.parameters(), lr = 0.001)

In [7]:
NUM_EPOCHS = 10 

for epoch in tqdm(range(1, NUM_EPOCHS +1), total=NUM_EPOCHS) : 
    train_loss = 0 
    val_loss = 0
    encoder.train(), decoder.train()
    for input, target in train_loader :
        input, target = input.to(device), target.to(device) 
        memory = encoder(input) 
        prediction, cross_attn = decoder(memory, target)

        loss = loss_fn(prediction, target[:, 1:, :]) 
        train_loss += loss.item()
        loss.backward()
        e_optim.step(), d_optim.step()
        e_optim.zero_grad(), d_optim.zero_grad() 

    # encoder.eval(), decoder.eval()
    # with torch.no_grad() :
    #     for input, target in val_loader :
    #         input, target = input.to(device), target.to(device) 
    #         memory = encoder(input) 
    #         prediction = decoder(memory, target)

    #         loss = loss_fn(prediction, target[:, 1:, :]) 
    #         val_loss += loss.item()
    print(f'Epoch {epoch} -- Train Loss: {train_loss / len(train_loader):.4f} -- Val Loss: {val_loss / len(val_loader):.4f}')

  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (21) must match the size of tensor b (20) at non-singleton dimension 3

In [32]:
prediction.shape



torch.Size([64, 21, 3])